In [1]:
import os

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gymnasium as gym

import gym_env
from utils import decision_policy, woodbury
from utils_render import plot_decision_prob
from models import SR_IS

In [2]:
# Set the random seed for NumPy
seed = 182
np.random.seed(seed)

# Hyperparams
reward = -0.1
alpha = 0.55
beta = 1.0
_lambda = 1.0
num_steps = 80000
decay = False
decay_params = [0.99, 150]

# For plotting
prob_locs = [12, 4, 6]
colors = [3, 2, 4]

# Save dir
save_dir = os.path.join('..', 'figures/')

## Learn DR-TD